In [55]:
from iminuit import describe, Minuit
import numpy as np

In [56]:
def model_func_1(x, a, b):
    return a * x + b

def model_func_2(x, c, d, e):
    return c * x - np.exp( d * x) + e


x_data = np.array([0,1,2,3,4,5,6,7,9,10])

In [59]:
def model_func_1(x, a, b):
    return a * x + b

def model_func_2(x, c, d, e):
    return c * x - np.exp( d * x) + e


x_data = np.array([0,1,2,3,4,5,6,7,9,10])

def NegativeLogLikelihood(model, **par):
    
    parameter_names = describe(model)[1:]  # assume that the first parameter is 'x'


    return - np.sum(np.log(model(x,**par)))



['a', 'b']

In [57]:
[name: None for name in param_names]

SyntaxError: invalid syntax (1201834975.py, line 1)

In [25]:
def set_var_if_None(var, x):
    if var is not None:
        return np.array(var)
    else: 
        return np.ones_like(x)
def compute_f(f, x, *par):
    
    try:
        return f(x, *par)
    except ValueError:
        return np.array([f(xi, *par) for xi in x])

In [44]:
xdata=[1,2,3,5,6]

In [50]:
compute_f(modelfunc, xdata, 1, 2)

TypeError: can only concatenate list (not "int") to list

In [51]:
class UnbinnedLH:  # override the class with a better one
    
    def __init__(self, f, data, _parameters=None, weights=None, bound=None, badvalue=-100000, extended=False, extended_bound=None,
                 extended_nint=100):
        
        if bound is not None:
            data = np.array(data)
            mask = (data >= bound[0]) & (data <= bound[1])
            data = data[mask]
            if (weights is not None) :
                weights = weights[mask]

        self.f = f  # model predicts PDF for given x
        self.data = np.array(data)
        self.weights = set_var_if_None(weights, self.data)
        self.bad_value = badvalue
        
        self.extended = extended
        self.extended_bound = extended_bound
        self.extended_nint = extended_nint
        if extended and extended_bound is None:
            self.extended_bound = (np.min(data), np.max(data))

        
        #self.func_code = make_func_code(describe(self.f)[1:])
        
        # Updated part: Use _parameters instead of func_code
        param_names = describe(self.f)[1:]  # assuming the first parameter is 'x'
        self._parameters = {name: None for name in param_names}

    def __call__(self, **par):  # par are a variable number of model parameters
        
        logf = np.zeros_like(self.data)
        
        # compute the function value
        f = compute_f(self.f, self.data, **par)
    
        # find where the PDF is 0 or negative (unphysical)        
        mask_f_positive = (f>0)

        # calculate the log of f everyhere where f is positive
        logf[mask_f_positive] = np.log(f[mask_f_positive]) * self.weights[mask_f_positive] 
        
        # set everywhere else to badvalue
        logf[~mask_f_positive] = self.bad_value
        
        # compute the sum of the log values: the LLH
        llh = -np.sum(logf)
        
        if self.extended:
            extended_term = integrate1d(self.f, self.extended_bound, self.extended_nint, **par)
            llh += extended_term
        
        return llh
    
    def default_errordef(self):
        return 0.5

In [52]:
xdata = [1,2,4,3,7,8,9,10]
ubll= UnbinnedLH(modelfunc, xdata)

In [53]:
from iminuit import Minuit

In [54]:
m=Minuit(ubll, a=1, b=2)

RuntimeError: a is not one of the parameters []

In [62]:
import numpy as np
from iminuit import Minuit, describe

class NegativeLogLikelihood:
    def __init__(self, model, x_data):
        """
        Initialize the NLL class.

        Parameters:
        - model: The model function to fit.
        - x_data: Data points (independent variable).
        """
        self.model = model
        self.x_data = x_data
        self.param_names = describe(model)[1:]  # Get parameter names dynamically

    def __call__(self, *params):
        """
        Compute the Negative Log-Likelihood (NLL).

        Parameters:
        - params: Model parameters, dynamically matched to parameter names.

        Returns:
        - Negative log-likelihood value.
        """
        # Map parameter values to their names
        param_dict = dict(zip(self.param_names, params))
        
        # Calculate model predictions
        y = self.model(self.x_data, **param_dict)
        
        # Check for invalid values in the model output
        if np.any(y <= 0):
            return np.inf  # Return a large value if PDF is invalid
        
        # Compute the negative log-likelihood
        nll = -np.sum(np.log(y))
        return nll

    @property
    def func_code(self):
        """
        Required by Minuit to identify parameter names.
        """
        class FuncCode:
            def __init__(self, names):
                self.co_varnames = names
                self.co_argcount = len(names)

        return FuncCode(self.param_names)

# Example models
def model_func_1(x, a, b):
    return a * x + b

def model_func_2(x, c, d, e):
    return c * x - np.exp(d * x) + e

# Data
x_data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 9, 10])

# Minimize dynamically
def minimize_nll(model, x_data, initial_guess, parameter_limits=None):
    """
    Minimize the Negative Log-Likelihood for a given model.

    Parameters:
    - model: The model function to fit.
    - x_data: Data points (independent variable).
    - initial_guess: Dictionary of initial guesses for model parameters.
    - parameter_limits: Dictionary of parameter limits (optional).

    Returns:
    - Minuit object after minimization.
    """
    # Initialize the NLL object
    nll = NegativeLogLikelihood(model, x_data)

    # Create Minuit
    m = Minuit(nll, **initial_guess)
    m.errordef = Minuit.LIKELIHOOD  # Set error definition for likelihood

    # Apply parameter limits if provided
    if parameter_limits:
        for param, limits in parameter_limits.items():
            m.limits[param] = limits

    # Minimize
    m.migrad()

    return m

# Example usage with model_func_1
initial_guess_1 = {"a": 1.0, "b": 1.0}
parameter_limits_1 = {"a": (0, None), "b": (-np.inf, np.inf)}
m1 = minimize_nll(model_func_1, x_data, initial_guess_1, parameter_limits_1)
print("Model 1 Results:")
print(m1.values)

# Example usage with model_func_2
initial_guess_2 = {"c": 1.0, "d": 1.0, "e": 1.0}
parameter_limits_2 = {"c": (0, None), "d": (-np.inf, np.inf), "e": (-10, 10)}
m2 = minimize_nll(model_func_2, x_data, initial_guess_2, parameter_limits_2)
print("Model 2 Results:")
print(m2.values)


Model 1 Results:
<ValueView a=1.0975227348822817e+40 b=1.1430922215589642e+40>
Model 2 Results:
<ValueView c=0.9999999999999999 d=1.0 e=1.0>
